In [1]:
# load in csv
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving train.csv to train.csv
User uploaded file "train.csv" with length 6690166 bytes


In [2]:
# load in dependencies
import pandas as pd

import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [633 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubun

In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tokens").getOrCreate()

In [4]:
from pyspark.ml.feature import Tokenizer

In [5]:
from pyspark.sql.functions import col, udf, isnan, count
from pyspark.sql.types import IntegerType

In [6]:
# read csv into datafram
raw_pet_df = pd.read_csv('train.csv')
raw_pet_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


Explore Data

In [7]:
# get column names
raw_pet_df.columns

Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')

In [8]:
# check for null values
raw_pet_df.isnull().sum()

Type                0
Name             1257
Age                 0
Breed1              0
Breed2              0
Gender              0
Color1              0
Color2              0
Color3              0
MaturitySize        0
FurLength           0
Vaccinated          0
Dewormed            0
Sterilized          0
Health              0
Quantity            0
Fee                 0
State               0
RescuerID           0
VideoAmt            0
Description        12
PetID               0
PhotoAmt            0
AdoptionSpeed       0
dtype: int64

In [9]:
raw_pet_df.count()

Type             14993
Name             13736
Age              14993
Breed1           14993
Breed2           14993
Gender           14993
Color1           14993
Color2           14993
Color3           14993
MaturitySize     14993
FurLength        14993
Vaccinated       14993
Dewormed         14993
Sterilized       14993
Health           14993
Quantity         14993
Fee              14993
State            14993
RescuerID        14993
VideoAmt         14993
Description      14981
PetID            14993
PhotoAmt         14993
AdoptionSpeed    14993
dtype: int64

In [10]:
# check datatypes
raw_pet_df.dtypes

Type               int64
Name              object
Age                int64
Breed1             int64
Breed2             int64
Gender             int64
Color1             int64
Color2             int64
Color3             int64
MaturitySize       int64
FurLength          int64
Vaccinated         int64
Dewormed           int64
Sterilized         int64
Health             int64
Quantity           int64
Fee                int64
State              int64
RescuerID         object
VideoAmt           int64
Description       object
PetID             object
PhotoAmt         float64
AdoptionSpeed      int64
dtype: object

Clean training data for model

Remove columns: Name, State, RescurerID, PetID since they are all identfication data. Keep PetID until cleaning can be done incase need something to merge on

In [11]:
#remove noisy data
pet_df = raw_pet_df.drop(["Name", "State", "RescuerID"], axis=1)
pet_df.head()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [12]:
#make sure no nulls
pet_df.isnull().sum()

Type              0
Age               0
Breed1            0
Breed2            0
Gender            0
Color1            0
Color2            0
Color3            0
MaturitySize      0
FurLength         0
Vaccinated        0
Dewormed          0
Sterilized        0
Health            0
Quantity          0
Fee               0
VideoAmt          0
Description      12
PetID             0
PhotoAmt          0
AdoptionSpeed     0
dtype: int64

In [13]:
#drop nulls in descripion since only a few rows of data will be lost (12)
pet_df.dropna(inplace=True)

In [14]:
pet_df.isnull().sum()

Type             0
Age              0
Breed1           0
Breed2           0
Gender           0
Color1           0
Color2           0
Color3           0
MaturitySize     0
FurLength        0
Vaccinated       0
Dewormed         0
Sterilized       0
Health           0
Quantity         0
Fee              0
VideoAmt         0
Description      0
PetID            0
PhotoAmt         0
AdoptionSpeed    0
dtype: int64

In [15]:
pet_df.count

<bound method DataFrame.count of        Type  Age  Breed1  ...      PetID  PhotoAmt  AdoptionSpeed
0         2    3     299  ...  86e1089a3       1.0              2
1         2    1     265  ...  6296e909a       2.0              0
2         1    1     307  ...  3422e4906       7.0              3
3         1    4     307  ...  5842f1ff5       8.0              2
4         1    1     307  ...  850a43f90       3.0              2
...     ...  ...     ...  ...        ...       ...            ...
14988     2    2     266  ...  dc0935a84       3.0              2
14989     2   60     265  ...  a01ab5b30       3.0              4
14990     2    2     265  ...  d981b6395       5.0              3
14991     2    9     266  ...  e4da1c9e4       3.0              4
14992     1    1     307  ...  a83d95ead       1.0              3

[14981 rows x 21 columns]>

In [16]:
quant_counts = pet_df['Quantity'].value_counts()
quant_counts

1     11556
2      1420
3       726
4       530
5       333
6       185
7        84
8        52
9        33
10       19
20       12
11       10
12        6
15        4
17        3
16        3
13        2
14        2
18        1
Name: Quantity, dtype: int64

bin 6 and greater

Quantity number defintions will now equal:
- 1 - 1 pet
- 2 - 2 pets
- 3 - 3 pets
- 4 - 4 pets
- 5 - 5 pets
- 6 - 6 or more

In [17]:
# Determine which values to replace if counts are less than ...?
replace_quant = list(quant_counts[quant_counts < 200].index)

# Replace in dataframe
for quant in replace_quant:
    pet_df.Quantity = pet_df.Quantity.replace(replace_quant,6)
    
# Check to make sure binning was successful
pet_df.Quantity.value_counts()

1    11556
2     1420
3      726
4      530
6      416
5      333
Name: Quantity, dtype: int64

In [18]:
pet_df['Fee'].value_counts()

0      12652
50       468
100      408
200      219
150      162
       ...  
210        1
99         1
330        1
75         1
599        1
Name: Fee, Length: 74, dtype: int64

make fee bins 0 (free), 1-50, 51-100, 101-150, 151-200, 201+

Fee_bins defintion
- 0 free
- 1 fee 1-50
- 2 fee 51-100
- 3 fee 101-150
- 4 fee 151-200
- 5 fee 201+

In [19]:
# Create the bins in which Data will be held
bins = [-1, 0, 50, 100, 150, 200, 5000]

# Create the names for the bins
group_names = [0, 1, 2, 3, 4, 5]

In [20]:
pet_df["Fee_bins"] = pd.cut(pet_df["Fee"], bins, labels=group_names)
pet_df.head(20)

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,Fee_bins
0,2,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,2
1,2,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0,0
2,1,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3,0
3,1,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2,3
4,1,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2,0
5,2,3,266,0,2,5,6,0,2,1,2,2,2,1,1,0,0,This is a stray kitten that came to my house. ...,d24c30b4b,2.0,2,0
6,2,12,264,264,1,1,0,0,2,3,2,2,3,1,1,300,0,anyone within the area of ipoh or taiping who ...,1caa6fcdb,3.0,1,5
7,1,0,307,0,2,1,2,7,2,1,2,2,2,1,6,0,0,Siu Pak just give birth on 13/6/10 to 6puppies...,97aa9eeac,9.0,3,0
8,2,2,265,0,2,6,0,0,2,2,2,2,2,1,1,0,0,"healthy and active, feisty kitten found in nei...",c06d167ca,6.0,1,0
9,2,12,265,0,2,1,7,0,2,2,3,3,3,1,1,0,0,"Very manja and gentle stray cat found, we woul...",7a0942d61,2.0,4,0


In [21]:
pet_df.isnull().sum()

Type             0
Age              0
Breed1           0
Breed2           0
Gender           0
Color1           0
Color2           0
Color3           0
MaturitySize     0
FurLength        0
Vaccinated       0
Dewormed         0
Sterilized       0
Health           0
Quantity         0
Fee              0
VideoAmt         0
Description      0
PetID            0
PhotoAmt         0
AdoptionSpeed    0
Fee_bins         0
dtype: int64

Find lenght of words in desciption column to allow for some assesment of description

In [22]:
dataframe = spark.read.csv("train.csv", sep=",", header=True)
dataframe.show(5)

+----+-----------+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------------------+--------+--------------------+---------+--------+-------------+
|Type|       Name|Age|Breed1|Breed2|Gender|Color1|Color2|Color3|MaturitySize|FurLength|Vaccinated|Dewormed|Sterilized|Health|Quantity|Fee|State|           RescuerID|VideoAmt|         Description|    PetID|PhotoAmt|AdoptionSpeed|
+----+-----------+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------------------+--------+--------------------+---------+--------+-------------+
|   2|     Nibble|  3|   299|     0|     1|     1|     7|     0|           1|        1|         2|       2|         2|     1|       1|100|41326|8480853f516546f6c...|       0|Nibble is a 3+ mo...|86e1089a3|     1.0|            2|
|   2|No Name Yet|  1|   265|     0|     1|     1|     2|     0|           2|       

In [23]:
# check for nulls https://stackoverflow.com/questions/44627386/how-to-find-count-of-null-and-nan-values-for-each-column-in-a-pyspark-dataframe
Dict_Null = {col:dataframe.filter(dataframe[col].isNull()).count() for col in dataframe.columns}
Dict_Null

{'AdoptionSpeed': 0,
 'Age': 0,
 'Breed1': 0,
 'Breed2': 0,
 'Color1': 0,
 'Color2': 0,
 'Color3': 0,
 'Description': 0,
 'Dewormed': 0,
 'Fee': 0,
 'FurLength': 0,
 'Gender': 0,
 'Health': 0,
 'MaturitySize': 0,
 'Name': 1257,
 'PetID': 0,
 'PhotoAmt': 0,
 'Quantity': 0,
 'RescuerID': 0,
 'State': 0,
 'Sterilized': 0,
 'Type': 0,
 'Vaccinated': 0,
 'VideoAmt': 0}

In [24]:
#pyspark changed empty to nan, so locate and drop https://stackoverflow.com/questions/52395986/remove-rows-from-dataframe-based-on-condition-in-pyspark
rows_to_delete=dataframe.filter(dataframe.Description == 'nan')
rows_to_delete.show()

+----+--------------------+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------------------+--------+-----------+---------+--------+-------------+
|Type|                Name|Age|Breed1|Breed2|Gender|Color1|Color2|Color3|MaturitySize|FurLength|Vaccinated|Dewormed|Sterilized|Health|Quantity|Fee|State|           RescuerID|VideoAmt|Description|    PetID|PhotoAmt|AdoptionSpeed|
+----+--------------------+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------------------+--------+-----------+---------+--------+-------------+
|   2|               Manja| 19|   285|     0|     2|     1|     6|     0|           3|        3|         1|       1|         2|     1|       1|  0|41401|12c51d7222986fbf0...|       0|        nan|5e4154870|     3.0|            1|
|   2|              Mickey|  2|   264|     0|     1|     1|     7|     0|           

In [25]:
dataframe = dataframe.join(rows_to_delete, on="PetID", how='left_anti')
dataframe.show(100)

+---------+----+--------------------+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------------------+--------+--------------------+--------+-------------+
|    PetID|Type|                Name|Age|Breed1|Breed2|Gender|Color1|Color2|Color3|MaturitySize|FurLength|Vaccinated|Dewormed|Sterilized|Health|Quantity|Fee|State|           RescuerID|VideoAmt|         Description|PhotoAmt|AdoptionSpeed|
+---------+----+--------------------+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------------------+--------+--------------------+--------+-------------+
|86e1089a3|   2|              Nibble|  3|   299|     0|     1|     1|     7|     0|           1|        1|         2|       2|         2|     1|       1|100|41326|8480853f516546f6c...|       0|Nibble is a 3+ mo...|     1.0|            2|
|6296e909a|   2|         No Name Yet|  1|   265|

In [26]:
# Tokenize sentences
tokenizer = Tokenizer(inputCol="Description", outputCol="words")
tokenizer

Tokenizer_48263e9c149a

In [27]:
tokenized_pet = tokenizer.transform(dataframe)
tokenized_pet.show(5)

+---------+----+-----------+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------------------+--------+--------------------+--------+-------------+--------------------+
|    PetID|Type|       Name|Age|Breed1|Breed2|Gender|Color1|Color2|Color3|MaturitySize|FurLength|Vaccinated|Dewormed|Sterilized|Health|Quantity|Fee|State|           RescuerID|VideoAmt|         Description|PhotoAmt|AdoptionSpeed|               words|
+---------+----+-----------+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------------------+--------+--------------------+--------+-------------+--------------------+
|86e1089a3|   2|     Nibble|  3|   299|     0|     1|     1|     7|     0|           1|        1|         2|       2|         2|     1|       1|100|41326|8480853f516546f6c...|       0|Nibble is a 3+ mo...|     1.0|            2|[nibble, is, a, 3...|


In [28]:
# Create a function to return the length of a list
def word_list_length(word_list):
    return len(word_list)

In [29]:
# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

In [30]:
# get count of words
token = tokenized_pet.withColumn('word_count', count_tokens(col("words")))
token.show(10)

+---------+----+--------------------+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------------------+--------+--------------------+--------+-------------+--------------------+----------+
|    PetID|Type|                Name|Age|Breed1|Breed2|Gender|Color1|Color2|Color3|MaturitySize|FurLength|Vaccinated|Dewormed|Sterilized|Health|Quantity|Fee|State|           RescuerID|VideoAmt|         Description|PhotoAmt|AdoptionSpeed|               words|word_count|
+---------+----+--------------------+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------------------+--------+--------------------+--------+-------------+--------------------+----------+
|86e1089a3|   2|              Nibble|  3|   299|     0|     1|     1|     7|     0|           1|        1|         2|       2|         2|     1|       1|100|41326|8480853f516546f6c...|      

merge dataframes
word_count definition - lenght of words in description column so we can perform analysis on it 

In [31]:
#convert to pandas
token_df = token.toPandas()
token_df.head()

,PetID,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PhotoAmt,AdoptionSpeed,words,word_count
0,86e1089a3,2,Nibble,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,1.0,2,"[nibble, is, a, 3+, month, old, ball, of, cute...",69
1,6296e909a,2,No Name Yet,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,2.0,0,"[i, just, found, it, alone, yesterday, near, m...",23
2,3422e4906,1,Brisco,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,7.0,3,"[their, pregnant, mother, was, dumped, by, her...",69
3,5842f1ff5,1,Miko,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",8.0,2,"[good, guard, dog,, very, alert,, active,, obe...",25
4,850a43f90,1,Hunter,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,3.0,2,"[this, handsome, yet, cute, boy, is, up, for, ...",81


In [34]:
token_df.isnull().sum()

PetID               0
Type                0
Name             1257
Age                 0
Breed1              0
Breed2              0
Gender              0
Color1              0
Color2              0
Color3              0
MaturitySize        0
FurLength           0
Vaccinated          0
Dewormed            0
Sterilized          0
Health              0
Quantity            0
Fee                 0
State               0
RescuerID           0
VideoAmt            0
Description         0
PhotoAmt            0
AdoptionSpeed       0
words               0
word_count          0
dtype: int64

In [33]:
token_df.loc[token_df['word_count'].isnull()]

,PetID,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PhotoAmt,AdoptionSpeed,words,word_count


In [35]:
token_wc_df = token_df[['PetID','word_count']]
token_wc_df.head(10)

,PetID,word_count
0,86e1089a3,69
1,6296e909a,23
2,3422e4906,69
3,5842f1ff5,25
4,850a43f90,81
5,d24c30b4b,18
6,1caa6fcdb,78
7,97aa9eeac,20
8,c06d167ca,13
9,7a0942d61,45


In [36]:
token_wc_df.isnull().sum()

PetID         0
word_count    0
dtype: int64

In [37]:
merged_pet_df = pet_df.merge(token_wc_df, how='left', on='PetID')
merged_pet_df.head()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,Fee_bins,word_count
0,2,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,2,69.0
1,2,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0,0,23.0
2,1,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3,0,69.0
3,1,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2,3,25.0
4,1,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2,0,81.0


In [38]:
merged_pet_df.isnull().sum()

Type               0
Age                0
Breed1             0
Breed2             0
Gender             0
Color1             0
Color2             0
Color3             0
MaturitySize       0
FurLength          0
Vaccinated         0
Dewormed           0
Sterilized         0
Health             0
Quantity           0
Fee                0
VideoAmt           0
Description        0
PetID              0
PhotoAmt           0
AdoptionSpeed      0
Fee_bins           0
word_count       365
dtype: int64

In [39]:
#check for duplicate petIds - try and figure out why there are now nulls
merged_pet_df.duplicated().sum()

0

full stop - reorganize - issue happends on merge try token first then other cleaning 

In [ ]:
# drop fee column and description
cleaned_pet_df = merged_pet_df.drop(["Fee", "Description", "PetID"], axis=1)
cleaned_pet_df.head()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,VideoAmt,PhotoAmt,AdoptionSpeed,Fee_bins,word_count
0,2,3,299,0,1,1,7,0,1,1,2,2,2,1,1,0,1.0,2,2,69.0
1,2,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,2.0,0,0,23.0
2,1,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,7.0,3,0,69.0
3,1,4,307,0,2,1,2,0,2,1,1,1,2,1,1,0,8.0,2,3,25.0
4,1,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,3.0,2,0,81.0


In [ ]:
cleaned_pet_df.isnull().sum()

Type               0
Age                0
Breed1             0
Breed2             0
Gender             0
Color1             0
Color2             0
Color3             0
MaturitySize       0
FurLength          0
Vaccinated         0
Dewormed           0
Sterilized         0
Health             0
Quantity           0
VideoAmt           0
PhotoAmt           0
AdoptionSpeed      0
Fee_bins           0
word_count       365
dtype: int64

In [ ]:
cleaned_pet_df.loc[cleaned_pet_df['word_count'].isnull()]

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,VideoAmt,PhotoAmt,AdoptionSpeed,Fee_bins,word_count
121,1,12,206,0,1,1,6,7,2,2,1,2,2,1,1,0,1.0,2,5,NaN
134,1,32,307,307,3,1,4,5,2,1,1,1,1,1,2,0,5.0,4,0,NaN
141,2,2,265,0,2,6,7,0,2,2,1,1,2,1,2,0,4.0,2,0,NaN
143,2,4,265,247,3,1,3,7,2,2,2,2,2,1,5,0,5.0,2,0,NaN
185,1,36,307,0,1,3,0,0,2,2,3,3,2,1,1,0,3.0,4,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,1,12,307,0,2,1,2,0,2,1,1,1,1,1,1,0,1.0,1,0,NaN
14873,2,2,266,0,1,6,0,0,2,1,2,2,2,1,1,0,5.0,2,0,NaN
14897,2,3,289,306,1,6,7,0,2,2,2,1,2,1,1,0,5.0,2,0,NaN
14904,1,3,307,0,1,2,7,0,3,1,2,1,2,1,1,0,11.0,2,0,NaN


Export cleaned training data

In [ ]:
# from https://stackoverflow.com/questions/53898836/export-dataframe-as-csv-file-from-google-colab-to-google-drive/60733479
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
# export cleaned data/df to new csv
cleaned_pet_df.to_csv('cleaned_train.csv', index=False)
!cp cleaned_train.csv "drive/My Drive/"